In [1]:
import tensorflow as tf
import gpflow as gpf
import numpy as np
from scipy.stats import qmc
import matplotlib.pyplot as plt

In [2]:
from model.multi_task_gp import MultiTaskGP

In [3]:
def fh1(x):
    return np.power(6 * x - 2, 2) * np.sin(12 * x - 4) + 10


def fh2(x):
    return 1.5 * (x + 2.5) * np.sqrt(fh1(x))


def fh3(x):
    return 5 * x * x * np.sin(12 ** x)


def fl1(x):
    return 2 * (x + 2) * np.sqrt(fh1(x) - 1.3 * (np.power(6 * x - 2, 2) - 6 * x))


def fl2(x):
    return np.power(6 * x - 2, 2) * np.sin(8 * x - 4) + 10 - (np.power(6 * x - 2, 2) - 6 * x)


def fl3(x):
    return fh3(x) + (x * x * x * np.sin(3 * x - 0.5)) + 4 * np.cos(2 * x)


fh = [fh1, fh2, fh3]
fl = [fl1, fl2, fl3]

In [4]:
sampler = qmc.LatinHypercube(d=1)
sample_hf = sampler.random(n=6).transpose()[0]
sample_lf = sampler.random(n=10).transpose()[0]
sample_lf = np.concatenate((sample_hf, sample_lf))

In [5]:
x = sample_lf.reshape(sample_lf.shape[0], 1)

In [6]:
X_augmented = np.vstack((np.hstack((x, np.zeros_like(x))), np.hstack((x, np.ones_like(x))), np.hstack((x, np.ones_like(x) * 2))))
Y_augmented = np.vstack((np.hstack((fl1(x), np.zeros_like(x))), np.hstack((fl2(x), np.ones_like(x))), np.hstack((fl3(x), np.ones_like(x) * 2))))

In [7]:
output_dim = 3 # Number of outputs
rank = 1 # Rank of W
k1 = gpf.kernels.Matern32(1, active_dims=[0]) # Base kernel
coreg = gpf.kernels.Coregion(output_dim=output_dim, rank=rank, active_dims=[1]) # Coregion kernel
white = gpf.kernels.White()
coreg.W = np.random.rand(output_dim, rank) # Initialise the W matrix
kern = k1 * coreg + white

In [8]:
# This likelihood switches between Gaussian noise with different variances for each f_i:
lik = gpf.likelihoods.SwitchedLikelihood([gpf.likelihoods.Gaussian(), gpf.likelihoods.Gaussian(), gpf.likelihoods.Gaussian()])

# now build the GP model as normal
m = MultiTaskGP([X_augmented, Y_augmented], kernel=kern, likelihood=lik)
# Here we specify num_latent=1 to avoid getting two outputs when predicting as Y_augmented is 2-dimensional

In [9]:
def optimize_model_with_scipy(model):
    optimizer = gpf.optimizers.Scipy()
    optimizer.minimize(
        model.training_loss,
        variables=model.trainable_variables,
        method="l-bfgs-b",
        options={"disp": True, "maxiter": 1000},
    )
optimize_model_with_scipy(m)

NotImplementedError: in user code:

    File "c:\users\jackp\pycharmprojects\triumf_gaussian_process\venv\lib\site-packages\gpflow\optimizers\scipy.py", line 120, in _tf_eval  *
        loss, grads = _compute_loss_and_gradients(
    File "c:\users\jackp\pycharmprojects\triumf_gaussian_process\venv\lib\site-packages\gpflow\optimizers\scipy.py", line 223, in _compute_loss_and_gradients  *
        loss = loss_closure()
    File "c:\users\jackp\pycharmprojects\triumf_gaussian_process\venv\lib\site-packages\gpflow\models\training_mixins.py", line 60, in training_loss  *
        return self._training_loss()  # type: ignore
    File "c:\users\jackp\pycharmprojects\triumf_gaussian_process\venv\lib\site-packages\gpflow\models\model.py", line 54, in _training_loss  *
        return -(self.maximum_log_likelihood_objective(*args, **kwargs) + self.log_prior_density())
    File "C:\Users\jackp\PycharmProjects\triumf_gaussian_process\model\multi_task_gp.py", line 24, in maximum_log_likelihood_objecti